In [2]:
import os, sys
import glob
import time
from datetime import timedelta, datetime
import pandas as pd
from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
#setting offilne
plotly.offline.init_notebook_mode(connected=True)
import pyarrow.parquet as pq
import json
from gezi import tqdm
import gezi
import qgrid

In [3]:
dc = gezi.read_parquet('../input/train/part_30/context.parquet')

In [4]:
dc

,label,mod,mf,aver,sver,did,vid,prev,region,timestamp,index
0,1,8e420890750df2b77a71e106dd52b17a,3de56d4b5045a6e2430a6c6b605784f4,856984b4b943d983e1ecd063561ede2d,e49dc1e8e576ff35a07e1182f4e98c91,2b738d0aa7695636ab1728073fbd86f6,2150409768,3466833237,1419611539,1426939472,0
1,0,8e420890750df2b77a71e106dd52b17a,3de56d4b5045a6e2430a6c6b605784f4,856984b4b943d983e1ecd063561ede2d,e49dc1e8e576ff35a07e1182f4e98c91,2b738d0aa7695636ab1728073fbd86f6,479266760,0,1419611539,1426940141,1
2,1,8e420890750df2b77a71e106dd52b17a,3de56d4b5045a6e2430a6c6b605784f4,856984b4b943d983e1ecd063561ede2d,e49dc1e8e576ff35a07e1182f4e98c91,2b738d0aa7695636ab1728073fbd86f6,4276937968,0,1419611539,1426939580,2
3,0,8e420890750df2b77a71e106dd52b17a,3de56d4b5045a6e2430a6c6b605784f4,856984b4b943d983e1ecd063561ede2d,e49dc1e8e576ff35a07e1182f4e98c91,2b738d0aa7695636ab1728073fbd86f6,1339723998,1102423729,1419611539,1426939474,3
4,0,8e420890750df2b77a71e106dd52b17a,3de56d4b5045a6e2430a6c6b605784f4,856984b4b943d983e1ecd063561ede2d,e49dc1e8e576ff35a07e1182f4e98c91,2b738d0aa7695636ab1728073fbd86f6,2488612978,0,1419611539,1426939580,4
...,...,...,...,...,...,...,...,...,...,...,...
3324508,0,d6380a5e93a7e09610801ee26ede1e86,9de1665e6430fa58785027dbf9ab53e0,3c0d3cb9415e8312b13cda9d0f111230,21032c3a66d86aa1d99127d4b97cb548,4d37aa0475ef29b6169e52c43e5ade94,751548132,0,2079574584,1426949698,3324508
3324509,0,52fe823f4e241bf498d82b6afc7fbc64,9de1665e6430fa58785027dbf9ab53e0,c088e05e114ccfd7e8f93d86d22652be,ba195dd85894d6957bc61edca5ac7026,3cd428138b93cf3aebebccaeb649d971,3592760380,526526995,3004987730,1426943422,3324509
3324510,0,52fe823f4e241bf498d82b6afc7fbc64,9de1665e6430fa58785027dbf9ab53e0,c088e05e114ccfd7e8f93d86d22652be,ba195dd85894d6957bc61edca5ac7026,3cd428138b93cf3aebebccaeb649d971,4054701265,0,3004987730,1426943425,3324510
3324511,1,52fe823f4e241bf498d82b6afc7fbc64,9de1665e6430fa58785027dbf9ab53e0,c088e05e114ccfd7e8f93d86d22652be,ba195dd85894d6957bc61edca5ac7026,3cd428138b93cf3aebebccaeb649d971,543883306,3859789495,3004987730,1426943420,3324511


In [5]:
di = gezi.read_parquet('../input/train/part_30/item.parquet')

In [6]:
di

,vid,title_length,timestamp,stars,cid,class_id,is_intact,second_class,duration,ctr,vv
0,15819492,98,1407204002,"[2950753864, 3600550670, 3606342894, 333836411...",638675564,1413776458,2234347078,0,6954.200195,0.0725,1996.0
1,38297769,112,1425690005,"[3115812425, 2144664037, 1221620183, 325915182...",2168442617,1413776458,472867291,0,42.509998,0.0000,5832.0
2,68075555,54,1370912171,[1688233050],243169446,2993616479,2234347078,0,292.119995,0.0000,4.0
3,72551649,28,1412650983,[],196493883,3174145329,2234347078,0,882.900024,0.0000,212.0
4,73710072,60,1397102895,"[2160407336, 3745672243, 325393889, 3169431763]",2056941596,1413776458,3172394293,0,25.070000,0.0000,56.0
...,...,...,...,...,...,...,...,...,...,...,...
42944,4248901384,21,1422849565,"[3902504787, 1235418045, 3804658193]",4074303298,1820266297,2234347078,0,1387.569946,0.0000,5388.0
42945,4255628613,33,1413545288,[1134598214],1016220003,1820266297,2234347078,0,35.970001,0.0000,440.0
42946,4272496705,80,1387029762,"[3309131045, 2561243119, 184102567, 2880021111...",435329116,1820266297,472867291,0,115.540001,0.0000,104.0
42947,4283547666,58,1424684908,"[2142910698, 1316768238, 1853987563, 1984414365]",2753664459,470292959,2234347078,0,154.779999,0.0600,12628.0


In [34]:
d = dc[['did', 'vid']].merge(di[['vid', 'cid']], on='vid', how='left')

In [35]:
d

,did,vid,cid
0,2b738d0aa7695636ab1728073fbd86f6,2150409768,2952613629
1,2b738d0aa7695636ab1728073fbd86f6,479266760,1548184080
2,2b738d0aa7695636ab1728073fbd86f6,4276937968,1336529284
3,2b738d0aa7695636ab1728073fbd86f6,1339723998,1338855061
4,2b738d0aa7695636ab1728073fbd86f6,2488612978,4285820984
...,...,...,...
3324508,4d37aa0475ef29b6169e52c43e5ade94,751548132,705281731
3324509,3cd428138b93cf3aebebccaeb649d971,3592760380,3174894263
3324510,3cd428138b93cf3aebebccaeb649d971,4054701265,4161563643
3324511,3cd428138b93cf3aebebccaeb649d971,543883306,3241637345


In [44]:
d1 = d.groupby(['did', 'cid']).size().reset_index(name='did_cid_counts')

In [45]:
d1

,did,cid,did_cid_counts
0,0000aa68cb571181da9ce69d451e67c9,222589797,1
1,0000aa68cb571181da9ce69d451e67c9,2759972708,1
2,0000aa68cb571181da9ce69d451e67c9,2760954775,1
3,0000aa68cb571181da9ce69d451e67c9,2973786887,1
4,0000aa68cb571181da9ce69d451e67c9,3303815793,1
...,...,...,...
2283068,fffec4082de126350b3ae3743e359e42,2759972708,1
2283069,fffec4082de126350b3ae3743e359e42,3393552497,1
2283070,fffec4082de126350b3ae3743e359e42,3825831318,2
2283071,fffec4082de126350b3ae3743e359e42,3952999864,2


In [46]:
d2 = d.groupby('did').size().reset_index(name='did_counts')

In [47]:
d2

,did,did_counts
0,0000aa68cb571181da9ce69d451e67c9,6
1,00023cbd0728e89fb5ed2fed5fd83612,3
2,0002a1d83ecdb9cd70518fd350c5ace4,5
3,0002a9e5f68969e3d488699dfd1ac76c,182
4,0003c32bfde744bd2aabee5b18f9ced1,18
...,...,...
92290,fffcb12ac5b86cde96de249e5b8655fe,5
92291,fffcb7794e1e7496f687c52429fd8c05,6
92292,fffda6aa89e05634706b97f065976baf,9
92293,fffde0944591a1e0391fffd795ec6ec4,246


In [48]:
d3 =   

In [49]:
d3

,did,cid,did_cid_counts,did_counts
0,0000aa68cb571181da9ce69d451e67c9,222589797,1,6
1,0000aa68cb571181da9ce69d451e67c9,2759972708,1,6
2,0000aa68cb571181da9ce69d451e67c9,2760954775,1,6
3,0000aa68cb571181da9ce69d451e67c9,2973786887,1,6
4,0000aa68cb571181da9ce69d451e67c9,3303815793,1,6
...,...,...,...,...
2283068,fffec4082de126350b3ae3743e359e42,2759972708,1,18
2283069,fffec4082de126350b3ae3743e359e42,3393552497,1,18
2283070,fffec4082de126350b3ae3743e359e42,3825831318,2,18
2283071,fffec4082de126350b3ae3743e359e42,3952999864,2,18


In [50]:
d3['rate'] = d3.did_cid_counts / d3.did_counts

In [53]:
d3

,did,cid,did_cid_counts,did_counts,rate
0,0000aa68cb571181da9ce69d451e67c9,222589797,1,6,0.166667
1,0000aa68cb571181da9ce69d451e67c9,2759972708,1,6,0.166667
2,0000aa68cb571181da9ce69d451e67c9,2760954775,1,6,0.166667
3,0000aa68cb571181da9ce69d451e67c9,2973786887,1,6,0.166667
4,0000aa68cb571181da9ce69d451e67c9,3303815793,1,6,0.166667
...,...,...,...,...,...
2283068,fffec4082de126350b3ae3743e359e42,2759972708,1,18,0.055556
2283069,fffec4082de126350b3ae3743e359e42,3393552497,1,18,0.055556
2283070,fffec4082de126350b3ae3743e359e42,3825831318,2,18,0.111111
2283071,fffec4082de126350b3ae3743e359e42,3952999864,2,18,0.111111


In [229]:
dids = set(dc.did.values)

In [230]:
vids = set(dc.vid.values)

In [231]:
len(vids)

32477

In [232]:
dc['id'] = dc.did + '\t' + dc.vid.astype(str)

In [233]:
ids = set(dc.id)

In [234]:
du = gezi.read_parquet('../input/train/part_30/user.parquet')

In [235]:
du

,did,watch
0,ec51024cacd62b2f44a5dedeab3b3ddd,"[[1426951647, 392178405], [1426948166, 2329609..."
1,e955cbbb099892dc8f6f3fb594ced710,"[[1426959713, 3784139339], [1426958209, 818147..."
2,e8ae4e4849f8c9a1e113d2c8b2654532,"[[1426950645, 1083423540], [1426950539, 119057..."
3,0c185e11deabef240415a6d5596c6fcf,"[[1426937047, 2633810051], [1426907692, 348815..."
4,668f8f04e920af6005177ece3d06309c,"[[1426944769, 3392895176], [1426944676, 263381..."
...,...,...
92290,7e92b833c60c61731f41822804b4b757,"[[1426948745, 4212935773], [1426948260, 293125..."
92291,e71abbf6c2ee6992d5bc7c6f86f433d8,"[[1426945479, 4101685914], [1426945376, 415305..."
92292,5a7ee331f09ec8f87dc5c0dc1823928a,"[[1426921816, 4153053381], [1426921294, 309199..."
92293,3c9d39a68f7c58fde21a2bd9196f6d76,"[[1426911634, 1099256000], [1426910255, 311265..."


In [236]:
du['watches'] =  du.watch.apply(json.loads)

In [237]:
def gtime(timestamp):
  x = time.localtime(timestamp)
  return f'{x.tm_mon}-{x.tm_mday}'

In [238]:
now_time = gtime(dc.timestamp.values[0])

In [239]:
now_time

'3-21'

In [240]:
did_vids = []
for _, row in tqdm(du.iterrows(), total=len(du)):
  did = row['did']
  watches = row['watches']
  for timestamp, vid in watches:
    if gtime(timestamp) == now_time:
      id_ = did + str(vid)
      if vid in vids and id_ not in ids:
        did_vids += [(did, vid, timestamp)]

100%|██████████| 92295/92295 [01:05<00:00, 1279.14it/s]


In [241]:
len(did_vids)

642468

In [242]:
dids, vids, timestamps = zip(*did_vids)

In [243]:
dvs = pd.DataFrame({'did': dids, 'vid': vids, 'timestamp': timestamps})

In [244]:
dvs

,did,vid,timestamp
0,ec51024cacd62b2f44a5dedeab3b3ddd,392178405,1426951647
1,ec51024cacd62b2f44a5dedeab3b3ddd,23296098,1426948166
2,ec51024cacd62b2f44a5dedeab3b3ddd,4210227665,1426945310
3,ec51024cacd62b2f44a5dedeab3b3ddd,618856631,1426945259
4,ec51024cacd62b2f44a5dedeab3b3ddd,2633810051,1426945250
...,...,...,...
642463,89fb732ebbc2df0eb4928b4b006644c6,2182542307,1426939503
642464,89fb732ebbc2df0eb4928b4b006644c6,1907952065,1426938156
642465,89fb732ebbc2df0eb4928b4b006644c6,4286416539,1426938132
642466,89fb732ebbc2df0eb4928b4b006644c6,2872596680,1426930667


In [245]:
dvs['time_str'] = dvs.timestamp.apply(gtime)

In [246]:
dvs

,did,vid,timestamp,time_str
0,ec51024cacd62b2f44a5dedeab3b3ddd,392178405,1426951647,3-21
1,ec51024cacd62b2f44a5dedeab3b3ddd,23296098,1426948166,3-21
2,ec51024cacd62b2f44a5dedeab3b3ddd,4210227665,1426945310,3-21
3,ec51024cacd62b2f44a5dedeab3b3ddd,618856631,1426945259,3-21
4,ec51024cacd62b2f44a5dedeab3b3ddd,2633810051,1426945250,3-21
...,...,...,...,...
642463,89fb732ebbc2df0eb4928b4b006644c6,2182542307,1426939503,3-21
642464,89fb732ebbc2df0eb4928b4b006644c6,1907952065,1426938156,3-21
642465,89fb732ebbc2df0eb4928b4b006644c6,4286416539,1426938132,3-21
642466,89fb732ebbc2df0eb4928b4b006644c6,2872596680,1426930667,3-21


In [247]:
dc_vid = dc[['vid', 'prev']].drop_duplicates().reset_index(drop=True)

In [248]:
from collections import Counter

In [249]:
m = {}

In [250]:
for _, row in dc_vid.iterrows():
  vid, prev = row['vid'], row['prev']
  if vid not in m:
    m[vid] = Counter()
  m[vid][prev] += 1

In [251]:
x = dc_vid.groupby('vid', as_index=False).count()

In [252]:
dvs

,did,vid,timestamp,time_str
0,ec51024cacd62b2f44a5dedeab3b3ddd,392178405,1426951647,3-21
1,ec51024cacd62b2f44a5dedeab3b3ddd,23296098,1426948166,3-21
2,ec51024cacd62b2f44a5dedeab3b3ddd,4210227665,1426945310,3-21
3,ec51024cacd62b2f44a5dedeab3b3ddd,618856631,1426945259,3-21
4,ec51024cacd62b2f44a5dedeab3b3ddd,2633810051,1426945250,3-21
...,...,...,...,...
642463,89fb732ebbc2df0eb4928b4b006644c6,2182542307,1426939503,3-21
642464,89fb732ebbc2df0eb4928b4b006644c6,1907952065,1426938156,3-21
642465,89fb732ebbc2df0eb4928b4b006644c6,4286416539,1426938132,3-21
642466,89fb732ebbc2df0eb4928b4b006644c6,2872596680,1426930667,3-21


In [253]:
prevs = []
for vid in dc_vid.vid:
  prevs += [m[vid].most_common(1)[0][0]]

In [254]:
dc_vid = pd.DataFrame({'vid': dc_vid.vid.values, 'prev': prevs})

In [255]:
dc_vid = dc_vid.drop_duplicates()

In [256]:
dc_vid

,vid,prev
0,2150409768,3466833237
1,479266760,0
2,4276937968,0
3,1339723998,1102423729
4,2488612978,0
...,...,...
45452,4083184484,0
45453,228793491,0
45454,1669998837,0
45455,3069310096,0


In [257]:
x[x.prev > 1]

,vid,prev
15,2278692,18
19,2694418,2
22,2949344,2
23,2999674,3
29,3763182,5
...,...,...
32420,4286897644,2
32455,4291743790,7
32456,4292156692,2
32463,4293147903,2


In [258]:
x = dc_vid.groupby('vid', as_index=False).max()
x[x.vid==2278692]

,vid,prev
15,2278692,3342863855


In [260]:
x[x.vid==2278692]

,vid,prev
15,2278692,3342863855


In [261]:
dc[dc.vid==2278692].prev

1389       3342863855
1696       3342863855
4321                0
7451       3342863855
9632       3342863855
              ...    
3305134    3342863855
3308171    3342863855
3316841    1017039896
3319029    3342863855
3319765    3342863855
Name: prev, Length: 1866, dtype: int64

In [262]:
len(dc[(dc.vid==2278692)&(dc.prev==3342863855)])

1126

In [263]:
len(dc[(dc.vid==2278692)&(dc.prev==0)])

45

In [264]:
dc_vid.groupby('vid', as_index=False)

In [265]:
dvs = dvs.merge(dc_vid, on='vid', how='left')

In [266]:
dc_did = dc.drop(['label', 'index', 'prev', 'id', 'timestamp', 'vid'], axis=1)

In [267]:
dc_did = dc_did.drop_duplicates()

In [268]:
dc_did

,mod,mf,aver,sver,did,region
0,8e420890750df2b77a71e106dd52b17a,3de56d4b5045a6e2430a6c6b605784f4,856984b4b943d983e1ecd063561ede2d,e49dc1e8e576ff35a07e1182f4e98c91,2b738d0aa7695636ab1728073fbd86f6,1419611539
18,f0c1a2307e75ee051478a212480b7aa1,932a5cc56763396c6b205a63868d6316,58655385700ba140fd860b23e2e1e9f6,ba195dd85894d6957bc61edca5ac7026,fb26d2effd6f01a88827c383fc320330,3410890293
58,af993e75bac3fff599a67bb27c237936,9de1665e6430fa58785027dbf9ab53e0,58655385700ba140fd860b23e2e1e9f6,e49dc1e8e576ff35a07e1182f4e98c91,a809a4940178dd4782be6dc80610482a,2220845847
148,a6ca1ea902400ca9d12822805daf8dd2,9de1665e6430fa58785027dbf9ab53e0,58655385700ba140fd860b23e2e1e9f6,95f0d16099fcfd8dab74600e61279f78,14fbf91a56aebc6ccf5824761882d159,1903481033
165,2fc392ba25fde32ed714ab319e9f6509,9a6e6cc222b26cc083646e13c32730ab,650894f17eee3eb5102c6394c9d9df90,f653f737068af83a709746a60da2a0ab,ba9e9f49d27b4a98a4325248adbfdc3d,82355409
...,...,...,...,...,...,...
3324350,18029e9cc7ab17cd01d83277231bfa95,9de1665e6430fa58785027dbf9ab53e0,16a774b916ad35634238e46c2c06395c,ba195dd85894d6957bc61edca5ac7026,e2c4681a85e0cef9e6b2643e7c461121,2257395272
3324362,8b744550ccf25645f01582d458e678bb,9de1665e6430fa58785027dbf9ab53e0,ef432ed6ec2f953c3bc5bffcebec9800,ba195dd85894d6957bc61edca5ac7026,086ad888462736283982e897182f7086,1418882033
3324437,d97292c09687efc2d6abe8af814df12f,9de1665e6430fa58785027dbf9ab53e0,3c0d3cb9415e8312b13cda9d0f111230,ba195dd85894d6957bc61edca5ac7026,06c594b9e9005e6dde6e8a4b0b493a36,2214269418
3324484,d6380a5e93a7e09610801ee26ede1e86,9de1665e6430fa58785027dbf9ab53e0,3c0d3cb9415e8312b13cda9d0f111230,21032c3a66d86aa1d99127d4b97cb548,4d37aa0475ef29b6169e52c43e5ade94,2079574584


In [269]:
x = dc_did.groupby('did', as_index=False).first()

In [270]:
x

,did,mod,mf,aver,sver,region
0,0000aa68cb571181da9ce69d451e67c9,cd9ae53a622604a3a12081c84a7bcc86,9de1665e6430fa58785027dbf9ab53e0,58655385700ba140fd860b23e2e1e9f6,ba195dd85894d6957bc61edca5ac7026,4110730136
1,00023cbd0728e89fb5ed2fed5fd83612,6d676dbe9f32bf5817cbf6849dafb0c4,49b4350906323cb44f6ce51ab22f2379,856984b4b943d983e1ecd063561ede2d,e49dc1e8e576ff35a07e1182f4e98c91,3806913320
2,0002a1d83ecdb9cd70518fd350c5ace4,d6e65ce07d50bac6c6292115244fd024,9de1665e6430fa58785027dbf9ab53e0,58655385700ba140fd860b23e2e1e9f6,ba195dd85894d6957bc61edca5ac7026,3654562113
3,0002a9e5f68969e3d488699dfd1ac76c,a141a3b1c47cb695347e53788d14a3ed,932a5cc56763396c6b205a63868d6316,58655385700ba140fd860b23e2e1e9f6,dd6f4a6f053926caa40f783216e5f471,1532815531
4,0003c32bfde744bd2aabee5b18f9ced1,f7afac3f0473e4fcc20db842b2ec5979,fce8cc8ce0587de13c3fecbc13537746,17695a4d053f9f2f1c1effd4ec446ae3,21032c3a66d86aa1d99127d4b97cb548,3379628159
...,...,...,...,...,...,...
92290,fffcb12ac5b86cde96de249e5b8655fe,34f5e821adce212e5f807fb3d4c28d7d,9de1665e6430fa58785027dbf9ab53e0,58655385700ba140fd860b23e2e1e9f6,ba195dd85894d6957bc61edca5ac7026,3490651091
92291,fffcb7794e1e7496f687c52429fd8c05,987aab9626d1e0f70b26aac30c00ae8b,9de1665e6430fa58785027dbf9ab53e0,58655385700ba140fd860b23e2e1e9f6,ba195dd85894d6957bc61edca5ac7026,2962402349
92292,fffda6aa89e05634706b97f065976baf,977b00fb6cb7943a2868e4f4671a1a09,9de1665e6430fa58785027dbf9ab53e0,58655385700ba140fd860b23e2e1e9f6,21032c3a66d86aa1d99127d4b97cb548,2284033415
92293,fffde0944591a1e0391fffd795ec6ec4,2ca00be0e25c135b9ab22c754cc6fb84,fce8cc8ce0587de13c3fecbc13537746,58655385700ba140fd860b23e2e1e9f6,21032c3a66d86aa1d99127d4b97cb548,3806913320


In [271]:
dvs = dvs.merge(x, on='did', how='left')

In [272]:
dvs

,did,vid,timestamp,time_str,prev,mod,mf,aver,sver,region
0,ec51024cacd62b2f44a5dedeab3b3ddd,392178405,1426951647,3-21,3333306837,f3fbd948bc5fe794a147d3029cfa0326,fce8cc8ce0587de13c3fecbc13537746,58655385700ba140fd860b23e2e1e9f6,e49dc1e8e576ff35a07e1182f4e98c91,4182017499
1,ec51024cacd62b2f44a5dedeab3b3ddd,23296098,1426948166,3-21,2331831542,f3fbd948bc5fe794a147d3029cfa0326,fce8cc8ce0587de13c3fecbc13537746,58655385700ba140fd860b23e2e1e9f6,e49dc1e8e576ff35a07e1182f4e98c91,4182017499
2,ec51024cacd62b2f44a5dedeab3b3ddd,4210227665,1426945310,3-21,0,f3fbd948bc5fe794a147d3029cfa0326,fce8cc8ce0587de13c3fecbc13537746,58655385700ba140fd860b23e2e1e9f6,e49dc1e8e576ff35a07e1182f4e98c91,4182017499
3,ec51024cacd62b2f44a5dedeab3b3ddd,618856631,1426945259,3-21,0,f3fbd948bc5fe794a147d3029cfa0326,fce8cc8ce0587de13c3fecbc13537746,58655385700ba140fd860b23e2e1e9f6,e49dc1e8e576ff35a07e1182f4e98c91,4182017499
4,ec51024cacd62b2f44a5dedeab3b3ddd,2633810051,1426945250,3-21,4153053381,f3fbd948bc5fe794a147d3029cfa0326,fce8cc8ce0587de13c3fecbc13537746,58655385700ba140fd860b23e2e1e9f6,e49dc1e8e576ff35a07e1182f4e98c91,4182017499
...,...,...,...,...,...,...,...,...,...,...
642463,89fb732ebbc2df0eb4928b4b006644c6,2182542307,1426939503,3-21,0,61da50302753dfd8133fa0aecfb5197c,932a5cc56763396c6b205a63868d6316,58655385700ba140fd860b23e2e1e9f6,21032c3a66d86aa1d99127d4b97cb548,1418882033
642464,89fb732ebbc2df0eb4928b4b006644c6,1907952065,1426938156,3-21,931422262,61da50302753dfd8133fa0aecfb5197c,932a5cc56763396c6b205a63868d6316,58655385700ba140fd860b23e2e1e9f6,21032c3a66d86aa1d99127d4b97cb548,1418882033
642465,89fb732ebbc2df0eb4928b4b006644c6,4286416539,1426938132,3-21,747712286,61da50302753dfd8133fa0aecfb5197c,932a5cc56763396c6b205a63868d6316,58655385700ba140fd860b23e2e1e9f6,21032c3a66d86aa1d99127d4b97cb548,1418882033
642466,89fb732ebbc2df0eb4928b4b006644c6,2872596680,1426930667,3-21,0,61da50302753dfd8133fa0aecfb5197c,932a5cc56763396c6b205a63868d6316,58655385700ba140fd860b23e2e1e9f6,21032c3a66d86aa1d99127d4b97cb548,1418882033


In [273]:
dvs['label'] = 1

In [274]:
dvs['index'] = -1

In [275]:
dvs

,did,vid,timestamp,time_str,prev,mod,mf,aver,sver,region,label,index
0,ec51024cacd62b2f44a5dedeab3b3ddd,392178405,1426951647,3-21,3333306837,f3fbd948bc5fe794a147d3029cfa0326,fce8cc8ce0587de13c3fecbc13537746,58655385700ba140fd860b23e2e1e9f6,e49dc1e8e576ff35a07e1182f4e98c91,4182017499,1,-1
1,ec51024cacd62b2f44a5dedeab3b3ddd,23296098,1426948166,3-21,2331831542,f3fbd948bc5fe794a147d3029cfa0326,fce8cc8ce0587de13c3fecbc13537746,58655385700ba140fd860b23e2e1e9f6,e49dc1e8e576ff35a07e1182f4e98c91,4182017499,1,-1
2,ec51024cacd62b2f44a5dedeab3b3ddd,4210227665,1426945310,3-21,0,f3fbd948bc5fe794a147d3029cfa0326,fce8cc8ce0587de13c3fecbc13537746,58655385700ba140fd860b23e2e1e9f6,e49dc1e8e576ff35a07e1182f4e98c91,4182017499,1,-1
3,ec51024cacd62b2f44a5dedeab3b3ddd,618856631,1426945259,3-21,0,f3fbd948bc5fe794a147d3029cfa0326,fce8cc8ce0587de13c3fecbc13537746,58655385700ba140fd860b23e2e1e9f6,e49dc1e8e576ff35a07e1182f4e98c91,4182017499,1,-1
4,ec51024cacd62b2f44a5dedeab3b3ddd,2633810051,1426945250,3-21,4153053381,f3fbd948bc5fe794a147d3029cfa0326,fce8cc8ce0587de13c3fecbc13537746,58655385700ba140fd860b23e2e1e9f6,e49dc1e8e576ff35a07e1182f4e98c91,4182017499,1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
642463,89fb732ebbc2df0eb4928b4b006644c6,2182542307,1426939503,3-21,0,61da50302753dfd8133fa0aecfb5197c,932a5cc56763396c6b205a63868d6316,58655385700ba140fd860b23e2e1e9f6,21032c3a66d86aa1d99127d4b97cb548,1418882033,1,-1
642464,89fb732ebbc2df0eb4928b4b006644c6,1907952065,1426938156,3-21,931422262,61da50302753dfd8133fa0aecfb5197c,932a5cc56763396c6b205a63868d6316,58655385700ba140fd860b23e2e1e9f6,21032c3a66d86aa1d99127d4b97cb548,1418882033,1,-1
642465,89fb732ebbc2df0eb4928b4b006644c6,4286416539,1426938132,3-21,747712286,61da50302753dfd8133fa0aecfb5197c,932a5cc56763396c6b205a63868d6316,58655385700ba140fd860b23e2e1e9f6,21032c3a66d86aa1d99127d4b97cb548,1418882033,1,-1
642466,89fb732ebbc2df0eb4928b4b006644c6,2872596680,1426930667,3-21,0,61da50302753dfd8133fa0aecfb5197c,932a5cc56763396c6b205a63868d6316,58655385700ba140fd860b23e2e1e9f6,21032c3a66d86aa1d99127d4b97cb548,1418882033,1,-1


In [276]:
dvs = dvs.drop(['time_str'], 1)

In [277]:
dc = dc.drop(['id'], 1)

In [278]:
dall = pd.concat([dc, dvs])

In [279]:
dall

,aver,did,index,label,mf,mod,prev,region,sver,timestamp,vid
0,856984b4b943d983e1ecd063561ede2d,2b738d0aa7695636ab1728073fbd86f6,0,1,3de56d4b5045a6e2430a6c6b605784f4,8e420890750df2b77a71e106dd52b17a,3466833237,1419611539,e49dc1e8e576ff35a07e1182f4e98c91,1426939472,2150409768
1,856984b4b943d983e1ecd063561ede2d,2b738d0aa7695636ab1728073fbd86f6,1,0,3de56d4b5045a6e2430a6c6b605784f4,8e420890750df2b77a71e106dd52b17a,0,1419611539,e49dc1e8e576ff35a07e1182f4e98c91,1426940141,479266760
2,856984b4b943d983e1ecd063561ede2d,2b738d0aa7695636ab1728073fbd86f6,2,1,3de56d4b5045a6e2430a6c6b605784f4,8e420890750df2b77a71e106dd52b17a,0,1419611539,e49dc1e8e576ff35a07e1182f4e98c91,1426939580,4276937968
3,856984b4b943d983e1ecd063561ede2d,2b738d0aa7695636ab1728073fbd86f6,3,0,3de56d4b5045a6e2430a6c6b605784f4,8e420890750df2b77a71e106dd52b17a,1102423729,1419611539,e49dc1e8e576ff35a07e1182f4e98c91,1426939474,1339723998
4,856984b4b943d983e1ecd063561ede2d,2b738d0aa7695636ab1728073fbd86f6,4,0,3de56d4b5045a6e2430a6c6b605784f4,8e420890750df2b77a71e106dd52b17a,0,1419611539,e49dc1e8e576ff35a07e1182f4e98c91,1426939580,2488612978
...,...,...,...,...,...,...,...,...,...,...,...
642463,58655385700ba140fd860b23e2e1e9f6,89fb732ebbc2df0eb4928b4b006644c6,-1,1,932a5cc56763396c6b205a63868d6316,61da50302753dfd8133fa0aecfb5197c,0,1418882033,21032c3a66d86aa1d99127d4b97cb548,1426939503,2182542307
642464,58655385700ba140fd860b23e2e1e9f6,89fb732ebbc2df0eb4928b4b006644c6,-1,1,932a5cc56763396c6b205a63868d6316,61da50302753dfd8133fa0aecfb5197c,931422262,1418882033,21032c3a66d86aa1d99127d4b97cb548,1426938156,1907952065
642465,58655385700ba140fd860b23e2e1e9f6,89fb732ebbc2df0eb4928b4b006644c6,-1,1,932a5cc56763396c6b205a63868d6316,61da50302753dfd8133fa0aecfb5197c,747712286,1418882033,21032c3a66d86aa1d99127d4b97cb548,1426938132,4286416539
642466,58655385700ba140fd860b23e2e1e9f6,89fb732ebbc2df0eb4928b4b006644c6,-1,1,932a5cc56763396c6b205a63868d6316,61da50302753dfd8133fa0aecfb5197c,0,1418882033,21032c3a66d86aa1d99127d4b97cb548,1426930667,2872596680


In [280]:
dall.label.mean()

0.22779589819058876

In [281]:
len(dc)

3324513

In [ ]:
len(dall)

In [ ]:
len(dvs)

In [ ]:
dce = gezi.read_parquet('../input/eval/context.parquet')

In [ ]:
dce['time_str'] = dce.timestamp.apply(gtime)

In [ ]:
dce

In [ ]:
len(dce[dce.prev==0]) / len(dce)

In [ ]:
dvs

In [282]:
dall.label.mean()

0.22779589819058876